## dominates

In [150]:
from binary_tournament_mo_skel import *

class testDominates(Individual):
    def dominates(self, other):
        """
        Multi-objective domination comparison between self and other
          if self dominates other: return 1
          elif other dominates self: return -1
          else: non-domination return 0
        """
        X_DOMINATES_Y = 1
        NON_DOMINATION = 0
        X_DOMINATED_BY_Y = -1
        
        #Check if dominates.
        dominationFlag_current  = 0
        dominationFlag_previous = 0
        for idx,item in enumerate(self.state):
            dominationFlag_previous = dominationFlag_current
            
            if item > other.state[idx]:
                dominationFlag_current = X_DOMINATED_BY_Y
            elif item < other.state[idx]:
                dominationFlag_current = X_DOMINATES_Y
            else:
                dominationFlag_current = dominationFlag_current
            
            if dominationFlag_previous != NON_DOMINATION and dominationFlag_current != dominationFlag_previous:
                return NON_DOMINATION
            
        return dominationFlag_current

In [151]:
xState = [2,1]
yState = [2,1]

x = testDominates(state = xState)
y = testDominates(state = yState)


print(f"Domination check1:{x.dominates(y)}")

zState = [3,7]
z = testDominates(state = zState)
print(f"Domination check2:{x.dominates(z)}")

zStateNew = [1,0]
z.state = zStateNew
print(f"Domination check3:{x.dominates(z)}")

#Non-domination
zStateNew = [1,7]
z.state = zStateNew
print(f"Domination check4:{x.dominates(z)}")

#Non-Trivial Case, should be z dominates
xStateNew = [2,1,4]
zStateNew = [2,1,1]
x.state = xStateNew
z.state = zStateNew
print(f"Domination check5:{x.dominates(z)}")

#Non-Trivial Case, should be x dominates
xStateNew = [0.7733822822904126, 4.8329365427122175]
zStateNew = [0.6833347494625214, 3.0870252374890295]
x.state = xStateNew
z.state = zStateNew
print(f"Domination check6:{x.dominates(z)}")

#Non-Trivial Case, should be Non-domination
xStateNew = [0.6833347494625214, 3.0870252374890295]
zStateNew = [0.8400439702669426, 3.9478685901211836]
x.state = xStateNew
z.state = zStateNew
print(f"Domination check7:{x.dominates(z)}")



Domination check1:0
Domination check2:1
Domination check3:-1
Domination check4:0
Domination check5:-1
Domination check6:-1
Domination check7:1


## CompareRankAndCrowding

In [152]:

class testcompareRankandCrowding(testDominates):
    def compareRankAndCrowding(self, other):
        """"
        Compare two individuals using frontRank and crowding distance metric, prefer div
        larger the better.
        Note: Compare by front-rank first. If front-ranks are equal, then compare using
        using crowding metric.
        if self better than other: return 1
        elif self worse than other: return -1
        else: return 0
        """
        compareRankFlag = 0
        if self.frontRank < other.frontRank:
            return 1
        elif self.frontRank > other.frontRank:
            return -1
        else:
            #Crowding metrics since we want diversity.
            #Distance larger the better
            if self.crowdDist > other.crowdDist:
                return 1
            elif self.crowdDist < other.crowdDist:
                return -1
            else:
                return 0 
                

In [153]:
xRank = 4
yRank = 4
xCrowdDist = 2
yCrowdDist = 2
x = testcompareRankandCrowding(frontRank=xRank,crowdDist=xCrowdDist)
y = testcompareRankandCrowding(frontRank=yRank,crowdDist=yCrowdDist)

print(x.compareRankAndCrowding(y))



0


# Population Compute frontRank

In [154]:
def printINFO(population):

    print("--------------------------------[INFO]---------------------------------------------------")

    for individual in population.pop:

        print(f"Individual has: {individual.state} with FrontRank:{individual.frontRank}")    



In [155]:
class testComputeFrontRank(Population):
    def computeFrontRanks(self):
        """
        Compute non-dominated front ranks using NSGA-II front-ranking scheme
        NSGA-II front-ranking scheme:
                Assign front rank for every Individual in POP
        """
        X_DOMINATES_Y = 1
        NON_DOMINATION = 0
        X_DOMINATED_BY_Y = -1
        
        newPop = []
        pop = self.pop
        frontRankCnt = 1
        
        while pop:
            #Keep doing until pop goes empty            
            front = [pop[0]]
            
            #Find the Pareto Front list
            for individual in pop:
                for idx,frontIndividual in enumerate (front):
                    
                    if frontIndividual.dominates(individual)   == X_DOMINATES_Y:
                        if individual in front:
                            front.remove(individual)
                    
                    elif frontIndividual.dominates(individual) == X_DOMINATED_BY_Y:
                        front.remove(frontIndividual)
                        
                        if individual not in front:
                            front.append(individual)
                    
                    elif frontIndividual.dominates(individual) == NON_DOMINATION:
                        if individual not in front:
                            front.append(individual)
                
                # for fronts in front:
                    # print(f"{fronts.state}")
                # 
                # print("------------")
                
            
            #From front assign the ranking value to pop
            for frontIndividual in front:
                frontIndividual.frontRank = frontRankCnt
                
                if frontIndividual not in newPop:
                    newPop.append(frontIndividual)
                
                pop.remove(frontIndividual)
                
            frontRankCnt += 1
            
        self.pop = newPop
        

## Test ComputeFrontRank


In [156]:
def minExInitializerTest(popSize,prng):
    #x1, x2 range limits
    x1_min=0.1
    x1_max=1.0
    x2_min=0.0
    x2_max=5.0
    #generate random population of Individuals
    # based on MinEx benchmark
    population=[]
    i=0
    while i < popSize:
        x1=prng.uniform(x1_min,x1_max)
        x2=prng.uniform(x2_min,x2_max)
        obj1=x1
        obj2=(1+x2)/x1
        population.append(testcompareRankandCrowding([x1,x2], [obj1,obj2]))
        i+=1
    return population


In [157]:
from random import Random
POP_SIZE = 10

popSize=500
numGenerations=10
prng=Random()
prng.seed(456)

pop1 = minExInitializerTest(popSize=POP_SIZE,prng = prng)

pop = testComputeFrontRank(pop = pop1)

def printINFO(population):
    print("--------------------------------[INFO]---------------------------------------------------")
    for individual in population.pop:
        print(f"Individual has: {individual.state} with FrontRank:{individual.frontRank}")    

printINFO(pop)


--------------------------------[INFO]---------------------------------------------------
Individual has: [0.7733822822904126, 4.8329365427122175] with FrontRank:None
Individual has: [0.4916883897151668, 3.971498902496217] with FrontRank:None
Individual has: [0.6833347494625214, 3.0870252374890295] with FrontRank:None
Individual has: [0.8400439702669426, 3.9478685901211836] with FrontRank:None
Individual has: [0.897832808715531, 1.6322445679051534] with FrontRank:None
Individual has: [0.9879261149141814, 0.5746088580927078] with FrontRank:None
Individual has: [0.5307432593866855, 2.3372173697580307] with FrontRank:None
Individual has: [0.33680835244373514, 0.6329469453583081] with FrontRank:None
Individual has: [0.26655746666688385, 1.9523119179750492] with FrontRank:None
Individual has: [0.5688309353215479, 3.9279756895830014] with FrontRank:None


In [158]:
pop.computeFrontRanks()
printINFO(pop)

--------------------------------[INFO]---------------------------------------------------
Individual has: [0.897832808715531, 1.6322445679051534] with FrontRank:1
Individual has: [0.9879261149141814, 0.5746088580927078] with FrontRank:1
Individual has: [0.33680835244373514, 0.6329469453583081] with FrontRank:1
Individual has: [0.26655746666688385, 1.9523119179750492] with FrontRank:1
Individual has: [0.4916883897151668, 3.971498902496217] with FrontRank:2
Individual has: [0.5307432593866855, 2.3372173697580307] with FrontRank:2
Individual has: [0.6833347494625214, 3.0870252374890295] with FrontRank:3
Individual has: [0.5688309353215479, 3.9279756895830014] with FrontRank:3
Individual has: [0.7733822822904126, 4.8329365427122175] with FrontRank:4
Individual has: [0.8400439702669426, 3.9478685901211836] with FrontRank:4


# Binary Tournament Method

In [159]:
class testbinaryTournament(testComputeFrontRank):
    def binaryTournament(self, prng):
        """
        Multi-objective binary tournament operator based on non-domination front-ranking scheme.
        Input Parameters:
        prng: Random number generator (i.e., random.Random object)
        Note: Similar to single-objective implementation,
          - Tournament pairs should be randomly selected
          - All individuals from initial pop should participate in exactly 2 tournaments
        """
        # binary tournament
        indexList1=list(range(len(self.pop)))
        indexList2=list(range(len(self.pop)))
        prng.shuffle(indexList1)
        prng.shuffle(indexList2)
        # do not allow pop competition
        for i in range(len(self.pop)):
            if indexList1[i] == indexList2[i]:
                temp=indexList2[i]
                if i == 0:
                    indexList2[i]=indexList2[-1]
                    indexList2[-1]=temp
                else:
                    indexList2[i]=indexList2[i-1]
                    indexList2[i-1]=temp
        #compete
        newPop=[]
        for index1,index2 in zip(indexList1,indexList2):
           if self.pop[index1].frontRank < self.pop[index2].frontRank:
               newPop.append(copy.deepcopy(self.pop[index1]))
           elif self.pop[index1].frontRank > self.pop[index2].frontRank:
               newPop.append(copy.deepcopy(self.pop[index2]))
           else:
               rn= prng.random()
               if rn > 0.5:
                   newPop.append(copy.deepcopy(self.pop[index1]))
               else:
                   newPop.append(copy.deepcopy(self.pop[index2]))
                   
        # overwrite old pop with newPop
        self.pop=newPop


In [160]:
pop1 = minExInitializerTest(popSize=POP_SIZE,prng = prng)
pop = testbinaryTournament(pop = pop1)

printINFO(pop)


--------------------------------[INFO]---------------------------------------------------
Individual has: [0.7396098003872399, 4.678576166033787] with FrontRank:None
Individual has: [0.6218340471082628, 3.4916665284025896] with FrontRank:None
Individual has: [0.9627389701456113, 4.065143366895537] with FrontRank:None
Individual has: [0.16664506580535132, 2.665747405351623] with FrontRank:None
Individual has: [0.4266431543883523, 4.929818387671791] with FrontRank:None
Individual has: [0.7522545276564475, 3.2332280972341003] with FrontRank:None
Individual has: [0.8842449410007605, 1.0023597387351946] with FrontRank:None
Individual has: [0.8180937238613244, 0.6452579509351913] with FrontRank:None
Individual has: [0.5018195742483234, 3.4174012257470476] with FrontRank:None
Individual has: [0.43357951779331283, 4.325844236558557] with FrontRank:None


In [161]:
pop.computeFrontRanks()
printINFO(pop)

--------------------------------[INFO]---------------------------------------------------
Individual has: [0.16664506580535132, 2.665747405351623] with FrontRank:1
Individual has: [0.8180937238613244, 0.6452579509351913] with FrontRank:1
Individual has: [0.5018195742483234, 3.4174012257470476] with FrontRank:1
Individual has: [0.43357951779331283, 4.325844236558557] with FrontRank:1
Individual has: [0.6218340471082628, 3.4916665284025896] with FrontRank:2
Individual has: [0.4266431543883523, 4.929818387671791] with FrontRank:2
Individual has: [0.7522545276564475, 3.2332280972341003] with FrontRank:2
Individual has: [0.8842449410007605, 1.0023597387351946] with FrontRank:2
Individual has: [0.7396098003872399, 4.678576166033787] with FrontRank:3
Individual has: [0.9627389701456113, 4.065143366895537] with FrontRank:3


In [162]:
pop.binaryTournament(prng)
printINFO(pop)

--------------------------------[INFO]---------------------------------------------------
Individual has: [0.7522545276564475, 3.2332280972341003] with FrontRank:2
Individual has: [0.5018195742483234, 3.4174012257470476] with FrontRank:1
Individual has: [0.8180937238613244, 0.6452579509351913] with FrontRank:1
Individual has: [0.16664506580535132, 2.665747405351623] with FrontRank:1
Individual has: [0.43357951779331283, 4.325844236558557] with FrontRank:1
Individual has: [0.8180937238613244, 0.6452579509351913] with FrontRank:1
Individual has: [0.43357951779331283, 4.325844236558557] with FrontRank:1
Individual has: [0.5018195742483234, 3.4174012257470476] with FrontRank:1
Individual has: [0.8842449410007605, 1.0023597387351946] with FrontRank:2
Individual has: [0.4266431543883523, 4.929818387671791] with FrontRank:2


In [163]:
generations = 10
for _ in range(generations):
    pop.binaryTournament(prng)
    
printINFO(pop)

--------------------------------[INFO]---------------------------------------------------
Individual has: [0.16664506580535132, 2.665747405351623] with FrontRank:1
Individual has: [0.5018195742483234, 3.4174012257470476] with FrontRank:1
Individual has: [0.43357951779331283, 4.325844236558557] with FrontRank:1
Individual has: [0.5018195742483234, 3.4174012257470476] with FrontRank:1
Individual has: [0.16664506580535132, 2.665747405351623] with FrontRank:1
Individual has: [0.43357951779331283, 4.325844236558557] with FrontRank:1
Individual has: [0.43357951779331283, 4.325844236558557] with FrontRank:1
Individual has: [0.5018195742483234, 3.4174012257470476] with FrontRank:1
Individual has: [0.43357951779331283, 4.325844236558557] with FrontRank:1
Individual has: [0.43357951779331283, 4.325844236558557] with FrontRank:1
